In [3]:
using Molly
using Colors
try
    using GLMakie
catch e
    import Pkg; Pkg.add("GLMakie")
end


In [4]:
n_atomsO = 2
n_atomsH = n_atomsO *2

atomO_mass = 16.0u"g/mol"
atomH_mass = 1.0u"g/mol"

atomsO = [Atom(index=i, mass=atomO_mass, σ=0.3u"nm", ϵ=0.2u"kJ * mol^-1") for i in 1:n_atomsO];
atomsH = [Atom(index=i, mass=atomH_mass, σ=0.1u"nm", ϵ=0.2u"kJ * mol^-1") for i in (n_atomsO+1):(n_atomsO+n_atomsH)];
atoms = [atomsO;atomsH]; # this concatenates two vectors


In [5]:
boundary = RectangularBoundary(5.0u"nm", 5.0u"nm") ;

coords = place_atoms(n_atomsO, boundary; min_dist=0.3u"nm"); # Random placement without clashing
for i in 1:length(coords)
    push!(coords, coords[i] .+ [0.1, 0.0]u"nm")
    push!(coords, coords[i] .+ [0.0, 0.1]u"nm")
end

#bonds = InteractionList2Atoms(
#    collect(1:(n_atomsO)),              # First atom indices
#    collect((n_atomsO+1):n_atomsH),     # Second atom indices
#    [HarmonicBond(k=300_000.0u"kJ * mol^-1 * nm^-2", r0=0.1u"nm") for i in 1:(n_atomsO)],
#)
#specific_inter_lists = (bonds,);

In [6]:
#fene_k = 250.0u"kJ * mol^-1 * nm^-2"
#fene_r0 = 1.6u"nm"
#bonds = InteractionList2Atoms(
    #collect(1:(n_atomsO)),              # First atom indices
    #collect((n_atomsO+1):n_atomsH),     # Second atom indices
    #[FENEBond(k=fene_k, r0=fene_r0, σ=0.4u"nm", ϵ=25u"kJ * mol^-1") for _ in 1:n_atomsO],)
#specific_inter_lists = (bonds,)







In [7]:
temp = 300.0u"K";
velocitiesO = [random_velocity(atomO_mass, temp, dims = 2) for i in 1:n_atomsO];
velocitiesH = [random_velocity(atomH_mass, temp, dims = 2) for i in 1:n_atomsH];
velocities = [velocitiesO; velocitiesH];


lj = LennardJones(
    #cutoff=DistanceCutoff(5.0u"nm"),
    #use_neighbors=true,
)

LennardJones{false, NoCutoff, Int64, Int64, Unitful.FreeUnits{(kJ, nm^-1, mol^-1), 𝐋 𝐌 𝐍^-1 𝐓^-2, nothing}, Unitful.FreeUnits{(kJ, mol^-1), 𝐋^2 𝐌 𝐍^-1 𝐓^-2, nothing}}(NoCutoff(), false, true, 1, 1, kJ nm^-1 mol^-1, kJ mol^-1)

In [9]:
sys = System(
    atoms=atoms,
    coords=coords,
    boundary=boundary,
    velocities=velocities,
    pairwise_inters=(lj,),
    #specific_inter_lists=(bonds,),
    loggers=(
        #temp=TemperatureLogger(10),
        coords=CoordinateLogger(1, dims = 2),
    ),
)
#sim = VelocityVerlet(dt=0.002u"ps", coupling=AndersenThermostat(temp, 1.0u"ps"),)
sim = Langevin(dt=0.002u"ps", temperature=300.0u"K", friction=1.0u"ps^-1")
simulate!(sys, sim, 1000)

System with 6 atoms, boundary RectangularBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}[5.0 nm, 5.0 nm])

In [11]:
[potential_energy(sys), kinetic_energy(sys)]
#temperature(sys)

colors = distinguishable_colors(2, [RGB(1, 1, 1), RGB(0, 0, 0)]; dropseed=true)


visualize(
    sys.loggers.coords, 
    boundary, 
    "sim_2d.mp4",
    #connections=[(bonds.is[i], bonds.js[i]) for i in 1:length(bonds.is)],
    #connections=zip(bonds.is, bonds.js),
    colors=repeat(colors; inner=n_atomsO),
    
)



"sim_2d.mp4"

In [12]:
display("text/html", """<video autoplay controls><source src="sim_2d.mp4" type="video/mp4"></video>""")